In [1]:
import numpy as np
import scipy as sp
from .propagator import propagator_fft

ImportError: attempted relative import with no known parent package

In [ ]:
def filter_weight(prop_array, 
                  t_list, 
                  noise_op, 
                  trunc_freq=None, 
                  prop_array_superop_fft=None):

    """
    Given a qutip.propagator (converted to a numpy array) and a noise operator, 
    this function returns the frequencies, the filter strength at different frequencies,
    and the filter operators (superop) at different frequencies. The filter strength is 
    defined as in Eq:22 of the paper. The filter operators are defined as in Eq:12 of the paper.
    
    Parameters:

    prop_array              : numpy array
                              The propagator generated by qutip.propagator

    t_list                  : list
                              Propagator time values

    noise_op                : Qobj
                              The noise operator

    trunc_freq              : float
                              The frequency range to truncate the filter strength to

    prop_array_superop_fft  : numpy array
                              The discrete fourier transform of the closed-system
                              superoperator which acts on the vectorized density matrix 
                              (at specified frequencies)

    Returns:

    fk_list                 : numpy array
                              The frequency bins

    filter_strength         : numpy array
                              The filter strength at different frequencies
    
    filter_op_dag_filter_op          : numpy array
                              The filter operator superoperators at different frequencies    
    
    """
    
    #Create frequency bins with successive bin centers separated by 1/(t_list[1]-t_list[0])
    fk_list = np.fft.fftfreq(len(t_list)-1, t_list[1]-t_list[0])

    #Sort the frequencies in increasing order (the output of fft.fftfreq has negative frequencies last)
    argsort = np.argsort(fk_list)
    fk_list = fk_list[argsort]

    #Convert the noise_op to a numpy array & extract the dimensionality
    noise_op_array = noise_op.full()
    dimension = noise_op_array.shape[-1]

    #Check if the fft of U*U^\dag (U : propagator) is provided, else compute it
    if prop_array_superop_fft is None:
        fk_list, prop_array_superop_fft = propagator_fft(prop_array, t_list, trunc_freq=None)


    #Array of the filter operators at different frequencies (x_k in [Eq:12])
    filter_op = np.einsum('ijmkl,kl->ijm', prop_array_superop_fft, noise_op_array)

    #Array of the adjoint of the filter operators at different frequencies
    filter_op_dag = np.conjugate(np.swapaxes(filter_op, 1, 2))

    #x_k^\dag * x_k at different frequencies
    filter_op_dag_filter_op = np.einsum('ijk,ikl->ijl', filter_op_dag, filter_op)

    #Calculate the filter strength [Eq:22] at different frequencies
    filter_strength = (np.trace(filter_op_dag_filter_op, axis1=1, axis2=2)-abs(np.trace(filter_op, axis1=1, axis2=2))**2/dimension).real

    #Sort the filter operators and the filter strengths in increasing order of frequency
    argsort         = np.argsort(fk_list)
    fk_list         = fk_list[argsort]
    filter_strength = filter_strength[argsort]
    filter_op       = filter_op[argsort]

    if trunc_freq is not None:
        argwhere                = np.argwhere(fk_list <= trunc_freq[1]).transpose()[0]

        fk_list                 = fk_list[argwhere]
        filter_strength         = filter_strength[argwhere]
        filter_op               = filter_op[argwhere]

        argwhere                = np.argwhere(fk_list >= trunc_freq[0]).transpose()[0]
        fk_list                 = fk_list[argwhere]
        filter_strength         = filter_strength[argwhere]
        filter_op               = filter_op[argwhere]

    return fk_list, filter_strength.real, filter_op

In [ ]:
def filter_weight_progressive(prop_array, 
                              t_list_sub, 
                              N_expand, 
                              noise_op, 
                              trunc_freq=None):

    """
    For each time value in a list of time values, this function calculates the filter 
    strengths, filter operators and frequency bins at different frequencies.

    Each sub-interval is expanded N_expand times to calculate the filter strength at
    different frequencies.

    Parameters                  :

    prop_array                  : numpy array
                                  The propagator generated by qutip.propagator    

    t_list_sub                  : list of floats  
                                  List of time values

    N_expand                    : int
                                  Number of times to expand the each sub-interval in t_list_sub

    noise_op                    : Qobj
                                  The noise operator

    trunc_freq                  : list of floats
                                  The frequency range to truncate the filter strength to

    Returns                     :

    fk_list_prog                : numpy array
                                  The frequency bins at different time values

    filter_strength_prog        : numpy array
                                  The filter strength at different frequencies at different time values

    filter_op_prog              : numpy array
                                  The filter operators at different time values

    """
    t_list = np.linspace(t_list_sub[0], t_list_sub[-1], N_expand*(len(t_list_sub)-1)+1)

    fk_list_prog = np.array([None] * len(t_list_sub))
    filter_strength_prog = np.array([None] * len(t_list_sub))
    filter_op_prog = np.array([None] * len(t_list_sub))

    #Multiprocessing can be enabled here if this function is used in the end-to-end simulation
    for j in range(len(t_list_sub)):
        if t_list_sub[j] == np.amin(t_list_sub):
            continue
        
        fk_list, filter_strength, filter_op = filter_weight(prop_array[0:j*N_expand+1], t_list[0:j*N_expand+1],
                                                      noise_op, trunc_freq=trunc_freq)
        fk_list_prog[j]         = fk_list
        filter_strength_prog[j] = filter_strength
        filter_op_prog[j]       = filter_op

    return fk_list_prog, filter_strength_prog, filter_op_prog

In [ ]:
def kdshmap(filter_op, 
            Sfk_list, 
            t, 
            exp=True):

    """
    Calculates the second-order contribution to the Keldysh map or the self-energy at the final time, 
    t, based on the requirement given a list of filter operators and a list of noise spectral densities.
    Deals with just one noise operator.

    Parameters:

    filter_op : numpy array
                Array of the filter operators at different frequencies

    Sfk_list  : numpy array 
                List of noise spectral densities at different frequencies

    t         : numpy array 
                Time value at which the Keldysh map or the self-energy is calculated

    exp       : bool
                If True, the Keldysh map is returned, else the self-energy is returned.

    """

    if np.amax(t) != np.amin(t):
        t = np.amax(t) - np.amin(t)
    else:
        t = abs(np.amax(t))

    dimension     = filter_op[0].shape[-1]
    identity      = np.eye(dimension, dtype=complex)
    filter_op_dag = np.conjugate(np.swapaxes(filter_op, 1, 2))
    lindb         = np.einsum('ijk,ilm->ijklm', filter_op, filter_op_dag)

    filter_op_dag_filter_op = np.einsum('ijk,ikl->ijl', filter_op_dag, filter_op)

    lindb += -0.5 * np.einsum('ijk,lm->ijklm', filter_op_dag_filter_op, identity)
    lindb += -0.5 * np.einsum('jk,ilm->ijklm', identity, filter_op_dag_filter_op)
    lindb = np.einsum('ijklm,i->jklm', lindb, Sfk_list) * t
    lindb = np.einsum('jklm->jmkl', lindb)
    lindb = lindb.reshape(dimension*dimension, dimension*dimension)

    if exp is True:
        return sp.linalg.expm(lindb)
    else:
        return lindb

In [ ]:
def kdshmap_multi_n_ops(filter_op_list: list, 
                        Sfk_list_list: list, 
                        t):
    
    """
    Calculates the second order Keldysh map contribution at the final time, t, for a list of noise operators.
    Calls the kdshmap function for each noise operator and sums the contributions.
    
    Parameters:

    filter_op_list : list
                     List of filter operators from different noise operators.
                     Each element in the list is a numpy array of the filter operators at different frequencies.

    Sfk_list_list  : list
                     List of different noise spectral densities associated with the noise operators.
                        Each element in the list is a numpy array of the noise spectral densities at different frequencies.  

    t              : float
                     Time value at which the Keldysh map is calculated     
                     
    """

    dimension = filter_op_list[0][0].shape[-1]

    lindb = np.zeros((dimension*dimension, dimension*dimension), dtype=complex)
    
    for n_ in range(len(filter_op_list)):
        lindb += kdshmap(filter_op_list[n_], Sfk_list_list[n_], t, exp=False)

    return sp.linalg.expm(lindb)